# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project
     WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Task 1

Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In order to solve this problem, we need information about the song and the session. The columns `session_id`, `item_in_session`, `artist_name`, `song`, `song_length` will be needed to achieve this query. The primary key will be `(session_id, item_in_session)` which uniquely identifies each row. It is made up of the partition key - `session_id` and the clustering column - `item_in_session`. The partion key determines how the data will be distributed across the nodes while the clustering column determines the sorting order.

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_songs"
query = query + "(session_id int, item_in_session int, artist_name varchar, song varchar, song_length decimal, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)       

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (session_id, item_in_session, artist_name, song, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "select artist_name, song, song_length from session_songs WHERE session_id=338 and item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist_name, row.song, row.song_length])

print(t)

+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


## Task 2

Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
for userid = 10, sessionid = 182

In order to achieve this query, we need information about the song, the user who listened to the song and the session. The appropriate column here consists of `user_id`, `session_id`, `item_in_session`, `artist_name`, `song`, `first_name` and `last_name`, `song`, `first_name`, `last_name`. The primary key is `((user_id, session_id), item_in_session)` which uniquely identifies each row. The primary key is made up of both the partition key and the clustering column. The partition key is what determines how the data is distributed across nodes in the Cassandra cluster. Here, the partition key is `(user_id, session_id)`. This is a composite partition key meaning the data will be partitioned based on the combination of both `user_id` and `session_id`.

Finally, the clustering column determines how data is sorted within the partition. The clustering column here is `item_in_session` so that data is sorted by `item_in_session` in ascending order within each partition.

In [11]:
query = "CREATE TABLE IF NOT EXISTS user_music_history"
query = query + """(user_id int, session_id int,  item_in_session int, artist_name varchar, song varchar, first_name varchar, last_name varchar, 
PRIMARY KEY ((user_id, session_id), item_in_session))
WITH CLUSTERING ORDER BY (item_in_session ASC)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)   

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_music_history (user_id, session_id, item_in_session, artist_name, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select artist_name, song, first_name, last_name from user_music_history WHERE session_id=182 and user_id=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


for row in rows:
    print (row.artist_name, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Task 3

Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'                 

The query is asking for user names (first and last) of all users who have listened to a particular song. So we need information about the user and the songs. Columns `song`, `user_id`, `first_name` and `last_name` are all necessary to solve this query. The primary key is `(song, user_id)` and the partition key is the `song` so that data is distributed across nodes in the Cassandra cluster by songs

In [14]:
query = "CREATE TABLE IF NOT EXISTS song_listeners"
query = query + """(song varchar, user_id int, first_name varchar, last_name varchar, 
PRIMARY KEY (song, user_id))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
query = "select first_name, last_name from song_listeners WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

t = PrettyTable(['First Name', 'Last Name'])
for row in rows:
    t.add_row([row.first_name, row.last_name])

print(t)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [17]:
query = "DROP table session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP table user_music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP table song_listeners"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()